In [31]:
import xml.etree.ElementTree as xml
from glob import glob


In [43]:
data_path = r"../data/raw/*"

In [44]:
path = glob(data_path)
path[0]

'../data/raw/subject7072.xml'

In [52]:
path1 = path[0]
for p in path:
    root = xml.parse(p).getroot()
    print(len(root.findall("WRITING/TEXT")))
    


31
19
1071
233
219
42
49
273
62
581
173
73
220
1051
886
347
64
12
1311
96
332
15
605
91
90
53
198
18
36
52
360
1976
204
767
123
134
997
96
571
31
189
902
803
1037
48
1432
61
562
180
180
25
187
42
122
128
35
634
193
179
366
86
83
1026
82
166
51
148
898
168
357
472
1999
441
25
234
854
972
14
294
17
12
677
30
277
162
30
542
1668
29
163
1999
23
223
99
64
145
38
107
290
388
29
807
882
198
1088
380
1133
1294
363
708
223
55
1496
793
62
1095
135
1270
822
158
11
166
384
822
941
154
11
506
413
251
20
1156
23
1425
16
1407
25
72
12
80
162
1216
42
1036
1315
1215
242
123
464
285
235
938
259
159
590
163
463
1012
1114
354
1987
1131
902
45
26
71
35
539
1856
22
1771
286
22
1003
1182
996
120
851
1318
1475
1032
1028
494
473
117
84
554
65
17
1279
11
239
36
1991
159
573
14
1029
505
58
1019
124
1079
169
171
84
37
589
13
307
36
24
237
101
242
130
136
1338
93
19
19
211
20
269
167
1151
39
119
28
1260
232
42
316
212
1996
18
36
288
307
420
1047
146
82
215
21
27
453
1112
42
14
188
739
560
807
106
125
1143
1151
107